In [ ]:
%matplotlib -l

In [8]:
import random
from operator import and_, not_, or_
import functools
import inspect
import colorsys
import math
import copy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.attrs import LEMMA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from freqer import *
from palettes import *
from qualmath import *
%matplotlib qt

In [9]:
spacy.require_gpu()

True

In [10]:
nlp=spacy.load('en_core_web_lg')

In [ ]:
# maybe at some point explicitly train a lemmatizer on novels.

In [ ]:
nlp.max_length = 2000000
copperfield_text = open('copperfield.txt').read()
copperfield = nlp(copperfield_text)

In [ ]:
obelisk_text = open('obelisk_gate.txt', encoding='utf-8').read()
obelisk = nlp(obelisk_text)

In [ ]:
northanger_text = open('./novels/northanger.txt').read()
northanger = nlp(northanger_text)

In [ ]:
kids_text = open('./novels/just_kids.txt').read()
kids = nlp(kids_text)

In [ ]:
bloody_text = open('./novels/bloody_chamber.txt').read()
bloody = nlp(bloody_text)

In [ ]:
spells_text = open('./3e_spells.txt').read()
spells = nlp(spells_text)

In [ ]:
nlp.max_length = 2000000
got_text = open('./novels/got.txt').read()
got = nlp(got_text)

In [ ]:
nightwood_text = open('./novels/nightwood.txt').read()
nightwood = nlp(nightwood_text)

In [ ]:
for token in obelisk[1:1000]: print(token.text, token.pos_, token.dep_)

In [11]:
display_palette(aqua,bump=0.1)

NameError: name 'compand' is not defined

In [ ]:
grab_tokens(got, token_type="all", boring=False, pos_list=None)[5].pos_

In [ ]:
colorlist=draw_freq_chart(
    freq_chart_prop,
    0,
    'average_position',
    'frequency',
    palette_function = purple,
    background = [1,1,1],
    wheel = 'average_position',
    bump = 0
)

In [ ]:
plt.polar(trimmed_chart['frequency'],trimmed_chart['average_position'])

In [ ]:
%time lemmalist = lemmatize(grab_tokens(copperfield))
%time freq_chart = prep_freq_chart(copperfield,lemmalist)

In [ ]:
# can make this more efficient 
# even if you want to position _within_ all lemmas
# by having the lemmatizer only _position_ the lemmas of interest

%time lemmalist_prop = lemmatize(grab_tokens(copperfield,pos_list=['PROPN']))
%time freq_chart_prop = prep_freq_chart(copperfield,lemmalist_prop)

In [ ]:
freq_chart[0:30]

In [ ]:
trimwords = ['say','go','come','mr.']
trim_chart = freq_chart.loc[np.logical_not(freq_chart['word'].isin(trimwords))]

In [ ]:
smash_chart = copy.deepcopy(freq_chart)
smash_chart["frequency"] = smash_chart["frequency"]**0.7

In [ ]:
def dumb_sim(word,comp="cat"):
    return nlp(word).similarity(nlp(comp))
smash_chart["like_cat"] = smash_chart["word"].apply(dumb_sim)

In [ ]:
smash_chart["like_night"] = smash_chart["word"].apply(dumb_sim,comp="night")

In [ ]:
chart = draw_freq_chart(
    smash_chart,
    2,
    'like_night',
    'like_cat',
    palette_function = purple,
    background = [0,0,0],
    wheel = 'average_position',
    heat = 'average_position',
    bump = 0.5
)

In [ ]:
plt.plot(np.sin(smash_chart["frequency"]/np.max(smash_chart["frequency"])*100*math.pi))
plt.plot(smash_chart["frequency"])

In [ ]:
np.vstack(chart)

In [ ]:
plt.plot(np.vstack(chart)[:,0])

In [ ]:
axis = np.arange(0,1,0.01)
np.meshgrid(axis,axis)

In [ ]:
xx

In [ ]:


# plt.plot([0,3],'o',color=[0,0,0,1])

In [ ]:
display_palette(aqua)

In [ ]:
np.array(horrible).shapea

### proximity

In [ ]:
# to make this more meaningful 
# we need a more robust concept of statistical distance
# and actually a metric that's meaningful in this space is part of what we're thinking of
# more broadly

def average_distance(text_positions,item_1,item_2,absolute=True):
    distances = [
        np.min(text_positions[item_1] - pos)
        for pos in text_positions[item_2]
    ]
    if absolute:
        return [np.abs(np.average(distances)),np.std(distances)]
    return [(np.average(distances)),np.std(distances)]

In [ ]:
'janis' in freq_chart['word'].values

In [ ]:
def single_word_distance_frame(index_word,freq_chart,lemmalist,text_positions):
    word_frame = freq_chart.reindex(columns=['word','frequency'])
    distlist = [
        average_distance(
            text_positions,index_word,word
        ) for word in word_frame['word']
    ]
    avg_dist = pd.Series([stat[0] for stat in distlist],name="avg_dist")
    std_dist = pd.Series([stat[1] for stat in distlist], name="std_dist")
    return pd.concat([
        word_frame,avg_dist,std_dist],axis=1).sort_values(
        by='frequency',ascending=False
    ).reset_index(drop=True)
       

In [ ]:
text_positions=position_lemmas(copperfield)

In [ ]:
%time morning_frame = single_word_distance_frame('heart',freq_chart,lemmalist,text_positions)

In [ ]:
morning_frame.loc[morning_frame['word']=='happiness']

In [ ]:
draw_freq_chart(
    morning_frame[10:],
    50,
    'avg_dist',
    'frequency',
    palette_function = purple,
    background = [0,0,0],
    wheel = 'avg_dist',
    heat = 'frequency',
    bump = 0.2
)

In [ ]:
%time rimbaud_frame = single_word_distance_frame('rimbaud',freq_chart,lemmalist,text_positions)

In [ ]:
%time robert_frame = single_word_distance_frame('robert',freq_chart,lemmalist,text_positions)

In [ ]:
robert_frame.sort_values(by="avg_dist").loc[robert_frame['frequency'] > 5][0:50]

In [ ]:
freq_chart[50:100]

In [ ]:
trim_frame = rimbaud_frame.loc[rimbaud_frame['word'] != 'robert']

In [ ]:
draw_freq_chart(trim_frame,5,'avg_dist','std_dist',colorfunc=purple)

In [ ]:
def two_way_comp(text_positions,item_1,item_2,test_item):
    distances_1 = [
        np.min(text_positions[test_item] - pos)
        for pos in text_positions[item_1]
    ]
    distances_2 = [
        np.min(text_positions[test_item] - pos) 
        for pos in text_positions[item_2]
    ]
    return [
        np.average(distances_1),
        np.average(distances_2),
        np.std(distances_1),
        np.std(distances_2)
    ]

In [ ]:
def naive_two_word_distance_frame(
    index_word_1,index_word_2,freq_chart,lemmalist,text_positions
):
    word_frame = freq_chart.reindex(columns=['word','frequency','average_position'])
    distlist = [
        two_way_comp(
            text_positions,index_word_1,index_word_2,word
        ) for word in word_frame['word']
    ]
    avg_dist_1 = pd.Series([stat[0] for stat in distlist],name="avg_dist_1")
    avg_dist_2 = pd.Series([stat[1] for stat in distlist],name="avg_dist_2")
    std_dist_1 = pd.Series([stat[2] for stat in distlist], name="std_dist_1")
    std_dist_2 = pd.Series([stat[3] for stat in distlist], name="std_dist_2")
    naive_3_way = pd.Series(np.sqrt(avg_dist_1**2 + avg_dist_2**2), name = "naive_3_way")
    return pd.concat([
        word_frame,avg_dist_1,avg_dist_2,std_dist_1,std_dist_2,naive_3_way
    ],axis=1).sort_values(
        by='naive_3_way',ascending=True
        ).reset_index(drop=True)

In [ ]:
naive_two_word_distance_frame()

In [ ]:
robert_rimbaud_frame = naive_two_word_distance_frame(
    'robert','rimbaud',freq_chart,lemmalist,text_positions
)

In [ ]:

robert_rimbaud_frame["r"] = (
    robert_rimbaud_frame["avg_dist_1"] ** 2 + robert_rimbaud_frame["avg_dist_2"] ** 2
) ** 0.5
robert_rimbaud_frame["theta"] = (
    robert_rimbaud_frame["avg_dist_1"] / robert_rimbaud_frame["avg_dist_2"])
robert_rimbaud_frame["diff"] = (
    robert_rimbaud_frame["avg_dist_1"] - robert_rimbaud_frame["avg_dist_2"] 
)

robert_rimbaud_frame["pos_norm"] = (
    (robert_rimbaud_frame["average_position"] - len(kids))/len(kids)
)

In [ ]:
robert_rimbaud_frame

In [ ]:
draw_freq_chart(robert_rimbaud_frame,15,'diff','avg_dist_2',colorfunc=purple)

## chart comparisons

In [ ]:
def verbose_compare(token_1,token_2):
    return {
        'similarity':token_1.similarity(token_2),
        'word_1':token_1.text,
        'word_2':token_2.text
    }

In [ ]:
simlist = [
    verbose_compare(random.choice(tokenlist),random.choice(tokenlist)) 
    for n in range(2000)
]
simlist = sorted(simlist, key=itemgetter('similarity'), reverse=True)

In [ ]:
word=nlp('dog')
simlist = [
    verbose_compare(word,token)
    for token in tokenlist
]
simlist = sorted(simlist, key=itemgetter('similarity'), reverse=True)
fig,ax = plt.subplots()
ax.hist([item['similarity'] for item in simlist], bins=50)

word=nlp('cat')
simlist = [
    verbose_compare(word,token)
    for token in tokenlist
]
simlist = sorted(simlist, key=itemgetter('similarity'), reverse=True)
fig,ax = plt.subplots()
ax.hist([item['similarity'] for item in simlist], bins=50)

In [ ]:
from IPython.display import clear_output
import timeit

In [ ]:
# so doing the whole novel like this would take 25-27 hours on this laptop. 
# much faster if we parallelized it...
# maybe just the most frequent words?

i = 0
simdict = {}

start = timeit.default_timer()
run_size = len(tokenset)

for token in tokenset:
    i+=1
    print(str(i)+'/'+str(run_size)+':'+token.text)
    
    simdict[token] = [
        verbose_compare(token,word) for word in tokenset
    ]
    cur_time = (timeit.default_timer()-start)
    
    
    print('current runtime: ', cur_time)
    print('expected runtime: ', cur_time * (run_size/i))
    
    if i == 50:
        break
    

# all_similar = [
#     [verbose_compare(word,token) for token in tokenset] 
#     for word in tokenset
# ]

In [ ]:
simdict

In [ ]:
# compute a list of tokens -- or lemmas? by frequency.
# use that to weight the similarity calculation.

# we could also just go back to an idea about frequency / recency space.

## section for exploring k-means

In [ ]:
from sklearn.datasets import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);